Необходимые импорты

In [2]:
import pandas as pd

### **Задание 1**

Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

-   если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
-   для источников paid и email из России ставим ad;
-   для источников paid и email не из России ставим other;
-   все остальные варианты берём из traffic_source без изменений.


Загрузим датасет

In [9]:
df_log = pd.read_csv('./visit_log.csv', sep=';')

In [4]:
df_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   timestamp       18938 non-null  int64 
 1   visit_id        18938 non-null  object
 2   url             18938 non-null  object
 3   region          18938 non-null  object
 4   user_id         18938 non-null  object
 5   traffic_source  18938 non-null  object
dtypes: int64(1), object(5)
memory usage: 887.8+ KB


In [5]:
df_log['region'].unique()

array(['Russia', 'China', 'India', 'Ukraine', 'Belarus', 'Australia'],
      dtype=object)

Создадим и заполним столбец **'data_source'** согласно правилам

In [23]:
%%time

df_log.loc[df_log.traffic_source == 'yandex', 'source_type'] = 'organic'
df_log.loc[df_log.traffic_source == 'google', 'source_type'] = 'organic'
df_log.loc[(df_log.traffic_source.isin(['paid', 'email'])) &
           (df_log['region'] == 'Russia'), 'source_type'] = 'ad'
df_log.loc[(df_log.traffic_source.isin(['paid', 'email'])) &
           (df_log.region != 'Russia'), 'source_type'] = 'other'
df_log.loc[df_log['source_type'].isna(), 'source_type'] = df_log['traffic_source']
df_log.sample(10)

CPU times: user 28.9 ms, sys: 7.84 ms, total: 36.7 ms
Wall time: 34.8 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
5739,1550015235,63a68ba89c,https://host.ru/fd32c3d2de55949865,Belarus,eb02c3917f,direct,direct
2953,1549998574,da93e11af4,https://host.ru/6fda01ec57f23abc9e,Belarus,fcefeed6eb,google,organic
6188,1550017927,dcaf55f610,https://host.ru/ac2d98a5a63e7c6e28,Russia,6dd3906cb0,direct,direct
11229,1550047817,270febf1a4,https://host.ru/c8d9213a31839f9a3a,Russia,67c68701d6,direct,direct
1251,1549988158,d286dd1fa5,https://host.ru/3c19b4ef7371864fa3,Russia,cd91a91068,yandex,organic
15673,1550074396,b6119bfefe,https://host.ru/3004a8273caeef2867,Belarus,6509a02703,email,other
6953,1550022420,a7aaf02acc,https://host.ru/39fa884393666d45fc,Belarus,0099f22c43,email,other
10328,1550042400,ee67aec391,https://host.ru/2fcb223266880ccf57,Russia,8db30b9e02,direct,direct
17828,1550087540,c7fd5493db,https://host.ru/c2382eb3d6afc8d0f3,Belarus,f7db4b25c0,paid,other
6576,1550020179,5c1d7e0c94,https://host.ru/8e88d4703848cc0ec4,Russia,eb99c5bbfd,paid,ad


### **Задание 2**
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

Прочитайте содержимое файла с датафрейм.
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

Загрузим датасет

In [24]:
df_url = pd.read_csv('./URLs.txt')

In [25]:
df_url.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [26]:
df_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     93 non-null     object
dtypes: object(1)
memory usage: 876.0+ bytes


Отфильтруем новости по заданному шаблону, используя регулярное выражение

In [29]:
df_news_filtered = df_url.loc[df_url['url'].str.contains(r'/\d{8}-', regex=True)]
df_news_filtered.sample(10)

,url
80,/sport/36235520-v-britanii-razgorelsya-krupney...
40,/head/36038043-foto-otdyhayuschiy-obama-vzbudo...
44,/incidents/36173928-v-ekaterinburge-migranty-u...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
42,/video/36006794-pogibshaya-melamed-ostavila-pr...
67,/middleeast/36024757-v-sekretnoy-tyurme-v-siri...
62,/starlife/36053306-dzhigarhanyan-dal-shokiruyu...
22,/world/36075956-na-granitse-s-krymom-zahvachen...
15,/video/36012692-tramp-otkrestilsya-ot-rossii-i...
35,/business/36187789-v-rf-hotyat-povysit-nalogi-...
